# SWOT



In [ ]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv
from holoviews import opts
import matplotlib as mpl


In [ ]:
# read the} bounding box of the study area
val_pts = gpd.read_file(Path('../data/validation-locations/subset-validation-reservoirs-grand-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/subset-validation-reservoirs-grand.geojson'))

selected_reservoirs = [
    '0505', # dumboor. India
    '0810', # sirindhorn, Thailand.
    '0830', # Krasoew, Thailand.
    '0502', # Bhakra dam, India.
    '0518', # Bhadra, India.
    '0349', # vaaldam, South Africa.
    '0464', # Sterkspruit, South Africa.
    '0214', # Cijara, Spain
    '1498', # Toledo bend, US
    '0936', # Arrow, Canada
]
res_names = {
    '0505': 'Dumboor, In',
    '0810': 'Sirindhorn, Th',
    '0830': 'Krasoew, Th',
    '0502': 'Bhakra, In',
    '0518': 'Bhadra, In',
    '0349': 'Vaaldam, SA',
    '0464': 'Sterkspruit, SA',
    '0214': 'Cijara, Sp',
    '1498': 'Toledo Bend, US',
    '0936': 'Arrow, Ca'
}

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

val_res_pt.plot()

In [ ]:
start_date = '2024-01-01'
end_date = '2024-05-25'
RESERVOIR = '0505'
buffer_amt = 1000 # meters
RESERVOIR_NAME = res_names[RESERVOIR]
print(f'{RESERVOIR}: {RESERVOIR_NAME}')

In [ ]:
roi = val_res_poly[val_res_poly['tmsos_id']==RESERVOIR]
utm_crs = roi.estimate_utm_crs()
buffered_roi_utm = roi.to_crs(utm_crs).geometry.buffer(buffer_amt)
buffered_roi = buffered_roi_utm.to_crs('epsg:4326')
buffered_roi.plot()

## SWOT - determine pass number

In [ ]:
gd_track_fn = Path('../data/swot_orbit/swot_orbit.geojson')
gd_track = gpd.read_file(gd_track_fn)
gd_track.plot(facecolor='none', edgecolor='gray', lw=0.1, alpha=0.7)

In [ ]:
gd_track_subset = gd_track[gd_track.intersects(buffered_roi.geometry.values[0])]

roi_gdf = gpd.GeoDataFrame(buffered_roi).rename({0: 'geometry'}, axis=1)
roi_gdf = roi_gdf.set_geometry('geometry')
roi_gdf['name'] = RESERVOIR_NAME
roi_hv = roi_gdf.hvplot(geo=True, alpha=0.8)

cmap = mpl.colormaps['inferno']
gd_track_subset['color'] = [mpl.colors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, len(gd_track_subset.groupby('ID_PASS')))]
tracks_hv = gd_track_subset.hvplot(
    geo=True, tiles='OSM', color=gd_track_subset['color'], alpha=0.5, line_width=2,
    xlim=(int(buffered_roi.total_bounds[0])-1, int(buffered_roi.total_bounds[2])+1),
    ylim=(int(buffered_roi.total_bounds[1])-1, int(buffered_roi.total_bounds[3])+1),
    tools=['hover'], hover_cols=['ID_PASS']
)

(tracks_hv * roi_hv).opts(
    title=f"SWOT Ground Tracks over {RESERVOIR_NAME}", width=400
)

In [ ]:
gd_track_subset

In [ ]:
matches = gd_track[gd_track.intersects(buffered_roi.to_crs('epsg:4326').iloc[0])]
pass_ids = list(matches.ID_PASS)
print(f'Passes: {pass_ids}')

In [ ]:
from tmsswot.data import get_swot_id

swot_ds = get_swot_id(RESERVOIR, val_res_poly, '2022-01-01', '2024-05-30')
swot_ds

# Plot

In [ ]:
ds = swot_ds

ds['wse'].hvplot(
    x='x', y='y', rasterize=True, aspect='equal', clim=(80, 100) # , geo=True, # crs=ds.rio.crs
).opts(
    title='Water Surface Elevation (m)'
) + ds['wse_uncert'].hvplot(
    x='x', y='y', rasterize=True, clim=(0, 1), aspect='equal', cmap='RdYlBu_r' #  geo=True, crs=ds.rio.crs
).opts(
    title='Water Surface Elevation Uncertainty (m)'
)

In [ ]:
swot_ds['water_frac'].hvplot(
    x='x', y='y', rasterize=True, aspect='equal', clim=(0, 1) # , geo=True, # crs=swot_ds.rio.crs
).opts(
    title='Water Fraction'
) + swot_ds['water_frac_uncert'].hvplot(
    x='x', y='y', rasterize=True, clim=(0, 1), aspect='equal', cmap='RdYlBu_r' #  geo=True, crs=swot_ds.rio.crs
).opts(
    title='Water Fraction Uncertainty'
)

In [ ]:
swot_ds['water_area'].hvplot(
    x='x', y='y', rasterize=True, aspect='equal', clim=(0, 1e4) # , geo=True, # crs=swot_ds.rio.crs
).opts(
    title='Water Area'
) + swot_ds['water_area_qual'].hvplot(
    x='x', y='y', rasterize=True, clim=(0, 3), aspect='equal', cmap='RdYlBu_r' #  geo=True, crs=swot_ds.rio.crs
).opts(
    title='Water Area Quality (0=nominal,1=suspect,2=degraded,3=bad)'
)

## save

In [ ]:
save_fp = Path(f'../data/swot/{RESERVOIR}.nc')
swot_ds.to_netcdf(save_fp)

### Process selected reservoirs at once

In [ ]:
# for RESERVOIR in selected_reservoirs:
#     print(f'{RESERVOIR}')

#     val_res_poly.loc[val_res_poly['tmsos_id']==RESERVOIR]

#     buffer_amt = 0.05
#     buffered_roi = val_res_poly.loc[val_res_poly['tmsos_id']==RESERVOIR].copy()
#     buffered_roi['geometry'] = buffered_roi.buffer(buffer_amt)
#     bounds = ",".join([f"{x:.3f}" for x in buffered_roi.total_bounds])
#     print(bounds)

#     podaac_cmd = [
#         'podaac-data-downloader',
#         '-c', 'SWOT_L2_HR_Raster_100m_2.0',
#         '-d', '../data/swot/raw', f'-b={bounds}',
#         '--start-date', pd.to_datetime(start_date).strftime('%Y-%m-%dT%XZ'),
#         '--end-date', pd.to_datetime(end_date).strftime('%Y-%m-%dT%XZ'),
#     ]

#     # print(" ".join(podaac_cmd))

#     import subprocess

#     subprocess.run(podaac_cmd)

#     swot_ds = get_swot_id(RESERVOIR, '2022-01-01', '2024-05-30')
#     save_fp = Path(f'../data/swot/{RESERVOIR}.nc')
#     swot_ds.to_netcdf(save_fp)

In [ ]:
save_fp = Path(f'../data/swot/{RESERVOIR}.nc')
swot_ds.to_netcdf(save_fp)